In [1]:
import os
import pandas as pd
import glob
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import urllib

In [12]:
selection = 'marketinsights'
directory = "./utils/data/Scraped News/"
file_pattern = f"{selection}_data_*.csv"
files = glob.glob(os.path.join(directory, file_pattern))
print(files)
latest_file = max(files, key=os.path.getctime)
print(latest_file)

['./utils/data/Scraped News/marketinsights_data_2024-07-01.csv']
./utils/data/Scraped News/marketinsights_data_2024-07-01.csv


In [13]:
from utils.marketinsights import safe_literal_eval, process_tables
df = pd.read_csv(latest_file)
df.head()


,title,link,ticker,date,source,Article content,raw,tables,Time,Executives,Shareholders
0,Cordiant Digital Infrastructure Limited Announ...,https://www.marketscreener.com/quote/stock/COR...,CORD,02:00am,S&P Capital IQ,Cordiant Digital Infrastructure Limited announ...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'David Charles B...,2024-07-01 02:00:00,"[['David Charles Bonar', 'Treasurer', '53', '2...","[['Newton Investment Management Ltd.', '37,855..."
1,Certain Common Stock of Public Joint-Stock Com...,https://www.marketscreener.com/quote/stock/SOV...,SVCB,2024-06-30,S&P Capital IQ,Certain Common Stock of Public Joint-Stock Com...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Dmitry Vladimir...,2024-06-30 00:00:00,"[['Aleksey Valeryevich Panferov', 'Corporate O...",[]
2,Certain A Shares of Chengdu Qinchuan IoT Techn...,https://www.marketscreener.com/quote/stock/CHE...,688528,2024-06-30,S&P Capital IQ,Certain A Shares of Chengdu Qinchuan IoT Techn...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Ze Hua Shao', 1...",2024-06-30 00:00:00,"[['Jing Zhang', 'Chief Administrative Officer'...","[['Gfund Management Co., Ltd.', '1,454,100', '..."
3,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,https://www.marketscreener.com/quote/stock/ARA...,2222,Jun. 30,Reuters,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Amin Al-Nasser'...,2024-06-30 00:00:00,"[['Ahmad Al-Khowaiter', 'Chief Tech/Sci/R&D Of...","[['Government of Saudi Arabia', '198,890,120,0..."
4,Certain Employee Stock Options of Corebridge F...,https://www.marketscreener.com/quote/stock/COR...,CRBG,Jun. 29,S&P Capital IQ,Certain Employee Stock Options of Corebridge F...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Kevin Hogan', 1...",2024-06-29 00:00:00,"[['Adam Burk', 'Director/Board Member', '47', ...","[['AMERICAN INTERNATIONAL GROUP, INC.', '294,2..."


In [14]:
html_content = df['raw'][10]
def get_industry(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Locate all card-header divs
    card_headers = soup.find_all('div', class_='card-header')

    # Iterate through each card-header div and extract the required information
    for card_header in card_headers:
        # Get the title text from the card-header div
        title_text = card_header.get_text(strip=True)
        if title_text == 'Sector':
            # Find the next sibling card-content div
            card_content = card_header.find_next_sibling('div', class_='card-content')
            
            if card_content:
                # Get the sector details from the card-content div
                sector_details = list(map(lambda x: x.strip(), card_content.get_text().split('\n')))
                actual = ''
                for sector in sector_details:
                    if sector:
                        actual = sector
    return actual



In [15]:
html_content = df['raw'][13]

def get_contact_information(html_content):
    result = {}
    soup = BeautifulSoup(html_content, 'html.parser')
    # Find the div with class 'card-content'
    contact_info_div = soup.find_all('div', class_ = 'card mb-15')
    for div in contact_info_div:
        try:
        # Extract the paragraphs within this div
            contact_info_paragraphs = div.find_all('p', class_='m-0')
            if contact_info_paragraphs:
                
                company_name = contact_info_paragraphs[0].text
                address_line_1 = contact_info_paragraphs[1].text
                address_line_2 = contact_info_paragraphs[2].text
                phone_number = contact_info_paragraphs[3].text

                # Extract the website URL
                website = div.find('a', class_='m-0')['href']

            # Print the extracted information
            result['Company Name'] = company_name
            result['Address Line 1'] = address_line_1
            result['Address Line 2'] = address_line_2
            result['Phone Number'] = phone_number
            result['Website'] = website
        except:
            pass
    return result


{'Company Name': 'Housing & Urban Development Corp. Ltd.',
 'Address Line 1': 'Hudco Bhawan, Core-7-A India Habitat Centre',
 'Address Line 2': '110 003, New Delhi',
 'Phone Number': '+91 11 2464 9610',
 'Website': 'http://www.hudco.org'}

In [3]:
# Fetch the webpage
def get_country_mapping():
    url = "https://www.countrycode.org"
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table
    table = soup.find('table')

    # Extract the table headers
    headers = []
    for th in table.find_all('th'):
        headers.append(th.text.strip())

    # Extract the table rows
    rows = []
    for tr in table.find_all('tr'):
        cells = tr.find_all('td')
        row = [cell.text.strip() for cell in cells]
        if row:
            rows.append(row)

    # Create a DataFrame
    df = pd.DataFrame(rows, columns=headers)
    return df

In [4]:
mapping = get_country_mapping()
mapping.head()

,COUNTRY,COUNTRY CODE,ISO CODES,POPULATION,AREA KM2,GDP $USD
0,Afghanistan,93,AF / AFG,"29,121,286","647,500",20.65 Billion
1,Albania,355,AL / ALB,"2,986,952","28,748",12.8 Billion
2,Algeria,213,DZ / DZA,"34,586,184","2,381,740",215.7 Billion
3,American Samoa,1-684,AS / ASM,"57,881",199,462.2 Million
4,Andorra,376,AD / AND,"84,000",468,4.8 Billion


In [8]:
storage = {}
for i, row in mapping.iterrows():
    if row['COUNTRY CODE'] not in storage.keys():
        storage[row['COUNTRY CODE']] = row['COUNTRY']
    else:
        storage[row['COUNTRY CODE']] += f"/{row['COUNTRY']}"

print(storage)

{'93': 'Afghanistan', '355': 'Albania', '213': 'Algeria', '1-684': 'American Samoa', '376': 'Andorra', '244': 'Angola', '1-264': 'Anguilla', '672': 'Antarctica', '1-268': 'Antigua and Barbuda', '54': 'Argentina', '374': 'Armenia', '297': 'Aruba', '61': 'Australia/Christmas Island/Cocos Islands', '43': 'Austria', '994': 'Azerbaijan', '1-242': 'Bahamas', '973': 'Bahrain', '880': 'Bangladesh', '1-246': 'Barbados', '375': 'Belarus', '32': 'Belgium', '501': 'Belize', '229': 'Benin', '1-441': 'Bermuda', '975': 'Bhutan', '591': 'Bolivia', '387': 'Bosnia and Herzegovina', '267': 'Botswana', '55': 'Brazil', '246': 'British Indian Ocean Territory', '1-284': 'British Virgin Islands', '673': 'Brunei', '359': 'Bulgaria', '226': 'Burkina Faso', '257': 'Burundi', '855': 'Cambodia', '237': 'Cameroon', '1': 'Canada/United States', '238': 'Cape Verde', '1-345': 'Cayman Islands', '236': 'Central African Republic', '235': 'Chad', '56': 'Chile', '86': 'China', '57': 'Colombia', '269': 'Comoros', '682': 'Co

In [73]:

def label_country(contact_information, storage):
    country = 'Unknown'
    try:
        phone_number = contact_information['Phone Number'].replace('-', ' ').replace('+', '').replace('(', '').replace(')', '')
        extension = phone_number.split(' ')[0]
        try:
            country = storage[extension]
        except:
            try:
                country = storage[f'1-{extension}']
            except:
                if extension:
                    print(extension)
                    country = 'USA/Canada'
                else:
                    country = 'Unknown'
    except:
        country = 'Unknown'
    return country

df['Contact Information'] = df['raw'].apply(get_contact_information)
df['Country'] = df['Contact Information'].apply(lambda x: label_country(x, storage))
df.head()

514
877
416
617
804
804
408
703
703
703
703
425
408
847
763
408
647
781
702


,title,link,ticker,date,source,Article content,raw,tables,Time,Executives,Shareholders,Contact Information,Country
0,Cordiant Digital Infrastructure Limited Announ...,https://www.marketscreener.com/quote/stock/COR...,CORD,02:00am,S&P Capital IQ,Cordiant Digital Infrastructure Limited announ...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'David Charles B...,2024-07-01 02:00:00,"[['David Charles Bonar', 'Treasurer', '53', '2...","[['Newton Investment Management Ltd.', '37,855...",{'Company Name': 'Cordiant Digital Infrastruct...,USA/Canada
1,Certain Common Stock of Public Joint-Stock Com...,https://www.marketscreener.com/quote/stock/SOV...,SVCB,2024-06-30,S&P Capital IQ,Certain Common Stock of Public Joint-Stock Com...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Dmitry Vladimir...,2024-06-30 00:00:00,"[['Aleksey Valeryevich Panferov', 'Corporate O...",[],"{'Company Name': 'Sovcombank PJSC', 'Address L...",Kazakhstan/Russia
2,Certain A Shares of Chengdu Qinchuan IoT Techn...,https://www.marketscreener.com/quote/stock/CHE...,688528,2024-06-30,S&P Capital IQ,Certain A Shares of Chengdu Qinchuan IoT Techn...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Ze Hua Shao', 1...",2024-06-30 00:00:00,"[['Jing Zhang', 'Chief Administrative Officer'...","[['Gfund Management Co., Ltd.', '1,454,100', '...",{'Company Name': 'Chengdu Qinchuan IoT Technol...,Unknown
3,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,https://www.marketscreener.com/quote/stock/ARA...,2222,Jun. 30,Reuters,SAUDI ARAMCO - ARAMCO'S STRATEGIC GAS EXPANSIO...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...",{'Managers': {'Managers': {0: 'Amin Al-Nasser'...,2024-06-30 00:00:00,"[['Ahmad Al-Khowaiter', 'Chief Tech/Sci/R&D Of...","[['Government of Saudi Arabia', '198,890,120,0...","{'Company Name': 'Saudi Arabian Oil Co.', 'Add...",Unknown
4,Certain Employee Stock Options of Corebridge F...,https://www.marketscreener.com/quote/stock/COR...,CRBG,Jun. 29,S&P Capital IQ,Certain Employee Stock Options of Corebridge F...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","{'Managers': {'Managers': {0: 'Kevin Hogan', 1...",2024-06-29 00:00:00,"[['Adam Burk', 'Director/Board Member', '47', ...","[['AMERICAN INTERNATIONAL GROUP, INC.', '294,2...","{'Company Name': 'Corebridge Financial, Inc.',...",USA/Canada


In [84]:
for item in df[df['Country'] == 'Unknown']['raw']:
    if item:
        print(item)
        break

<!DOCTYPE html>
<html lang="en-US" prefix="og: https://ogp.me/ns#"><head><title>Chengdu Qinchuan IoT Technology Co.Ltd.: Shareholders Board Members Managers and Company Profile | CNE100003ZF5 | MarketScreener</title><link data-version="v000013" href="https://cdn.zonebourse.com/static/compiled-css/867694c2dd6b8fbc794549714fe80b27.min.css" rel="stylesheet"/><link href="https://code.jquery.com/" rel="preconnect"/><link href="https://securepubads.g.doubleclick.net/" rel="preconnect"/><link href="https://code.highcharts.com/" rel="preconnect"/><link href="https://www.googletagmanager.com/" rel="preconnect"/><link href="https://client.px-cloud.net/" rel="preconnect"/><link href="https://cdnjs.cloudflare.com/" rel="preconnect"/><link href="https://cdn.zonebourse.com/" rel="preconnect"/><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1, user-scalable=5, shrink-to-fit=no" name="viewport"/><meta content="Chengdu Qinchuan IoT Technology Co.Ltd., stock exchange, surperforma

In [89]:
print(df.iloc[60].ticker)
print(df.iloc[60].raw)

YNDX
<!DOCTYPE html>
<html lang="en-US" prefix="og: https://ogp.me/ns#"><head><title>Yandex N.V.: Shareholders Board Members Managers and Company Profile | NL0009805522 | MarketScreener</title><link data-version="v000013" href="https://cdn.zonebourse.com/static/compiled-css/867694c2dd6b8fbc794549714fe80b27.min.css" rel="stylesheet"/><link href="https://code.jquery.com/" rel="preconnect"/><link href="https://securepubads.g.doubleclick.net/" rel="preconnect"/><link href="https://code.highcharts.com/" rel="preconnect"/><link href="https://www.googletagmanager.com/" rel="preconnect"/><link href="https://client.px-cloud.net/" rel="preconnect"/><link href="https://cdnjs.cloudflare.com/" rel="preconnect"/><link href="https://cdn.zonebourse.com/" rel="preconnect"/><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1, user-scalable=5, shrink-to-fit=no" name="viewport"/><meta content="Yandex N.V., stock exchange, surperformance ratings, financial ratings, shareholders, manag

In [74]:
missing = set([514,
877,
416,
617,
804,
804,
408,
703,
703,
703,
703,
425,
408,
847,
763,
408,
647,
781,
702])

In [78]:
sorted(missing)

[408, 416, 425, 514, 617, 647, 702, 703, 763, 781, 804, 847, 877]

In [93]:
storage = {}
for i, row in mapping.iterrows():
    if row['COUNTRY CODE'] not in storage.keys():
        storage[row['COUNTRY CODE']] = row['COUNTRY']
    else:
        storage[row['COUNTRY CODE']] += f"/{row['COUNTRY']}"

print(list(storage.values()))

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia/Christmas Island/Cocos Islands', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada/United States', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Curacao/Netherlands Antilles', 'Cyprus', 'Czech Republic', 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands', 'Faroe Isla

In [131]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

# Load pre-trained model and tokenizer
#model_name = 'intfloat/e5-mistral-7b-instruct'
model_name = 'intfloat/multilingual-e5-large-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Function to get embeddings
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Function to find the most similar country
def find_country(address, countries):
    country_embeddings = get_embeddings(countries)
    address_embedding = get_embeddings([address])
    address_embedding = address_embedding.flatten() # Convert to 1-D array
    similarities = [1 - cosine(address_embedding, country_embedding) for country_embedding in country_embeddings]
    most_similar_country = countries[similarities.index(max(similarities))]
    return most_similar_country

# # Example address
address = "California Nevada"
countries = ['Germany', 'United States of America', 'United Kingdom', 'Ireland', 'Guernessey', 'China', 'Luxembourg', 'Turkey', 'Sweden', 'Singapore', 'Hong Kong', 'Taiwan', 'Dubai', 'India', 'Others']
region = ['Europe', 'North America', 'Asia', 'Oceania', 'Africa', 'South America']
country = find_country(address, countries)
print(f"The address belongs to: {country}")

The address belongs to: United States of America


In [144]:
test_set = df[df['Contact Information'] != {}]
#test_set['Country 2'] = test_set['Contact Information'].apply(lambda x: find_country(x['Address Line 2'], region))

In [134]:
for i, row in test_set[['Country', 'Country 2', 'Contact Information']].iterrows():
    if row['Country'] != row['Country 2']:
        if row['Country'] == 'Unknown':
            print(row['Country'],row['Country 2'])
            print(row['Contact Information'])
            print('********')

Unknown Asia
{'Company Name': 'Chengdu Qinchuan IoT Technology Co., Ltd.', 'Address Line 1': '931, South 4th Road Jingkai District', 'Address Line 2': '610100, Chengdu', 'Phone Number': '+', 'Website': 'http://www.cdqckj.com'}
********
Unknown Asia
{'Company Name': 'Saudi Arabian Oil Co.', 'Address Line 1': 'P. O. Box 5000', 'Address Line 2': '31311, Dhahran', 'Phone Number': '+', 'Website': 'http://www.aramco.com'}
********
Unknown Asia
{'Company Name': 'Nanjing Railway New Technology Co., Ltd.', 'Address Line 1': '19 Longtai Road Jiangbei New District', 'Address Line 2': '210061, Nanjing', 'Phone Number': '+', 'Website': 'http://www.njlew.com.cn'}
********
Unknown North America
{'Company Name': 'Verve Therapeutics, Inc.', 'Address Line 1': '201 Brookline Avenue Suite 601', 'Address Line 2': '02215, Boston', 'Phone Number': '+', 'Website': 'http://www.vervetx.com'}
********
Unknown Asia
{'Company Name': 'Huisen Household International Group Ltd.', 'Address Line 1': 'Huisen Road Daluo 

In [165]:
import re
import unicodedata

def remove_diacritics(input_str):
    # Normalize the string to decompose diacritics
    normalized_str = unicodedata.normalize('NFD', input_str)
    
    # Filter out the diacritic marks
    non_diacritic_str = ''.join(
        char for char in normalized_str if unicodedata.category(char) != 'Mn'
    )
    return non_diacritic_str

cities = pd.read_csv('worldcities.csv')
city_storage = {}

for i, row in cities.iterrows():
    city = remove_diacritics(row['city'])
    if city not in city_storage.keys():
        city_storage[city] = [row['country']]
    else:
        if row['country'] not in city_storage[city]:
            city_storage[city].append(row['country'])

def extract_city_names(lst):
    for i in range(len(lst) - 1, -1, -1):
        if re.search(r'\d', lst[i]):
            if i + 1 < len(lst):
                return ' '.join(lst[i+1:]).split(',')[-1].strip()

for i, row in test_set.iterrows():
    temp = row['Contact Information']['Address Line 2'].split(' ')
    city = extract_city_names(temp)
    try:
        country = city_storage[city]
    except:
        print(f'Error with {city}')


Error with St. Peter Port
Error with St. Peter Port
Error with Seongnam-si
Error with Seongnam-si
Error with North Chesterfield
Error with Saint-Étienne
Error with Sverige
Error with Thrissur
Error with St. Peter Port
Error with None
Error with Gurugram
Error with North Ryde
Error with Sangareddy
Error with Sepang
Error with Gurugram


In [166]:
city_storage

{'Tokyo': ['Japan'],
 'Jakarta': ['Indonesia'],
 'Delhi': ['India', 'United States'],
 'Guangzhou': ['China'],
 'Mumbai': ['India'],
 'Manila': ['Philippines'],
 'Shanghai': ['China'],
 'Sao Paulo': ['Brazil'],
 'Seoul': ['Korea, South'],
 'Mexico City': ['Mexico'],
 'Cairo': ['Egypt', 'United States'],
 'New York': ['United States'],
 'Dhaka': ['Bangladesh', 'India'],
 'Beijing': ['China'],
 'Kolkata': ['India'],
 'Bangkok': ['Thailand'],
 'Shenzhen': ['China'],
 'Moscow': ['Russia', 'United States'],
 'Buenos Aires': ['Argentina', 'Colombia', 'Costa Rica', 'Brazil'],
 'Lagos': ['Nigeria', 'Portugal', 'Greece'],
 'Istanbul': ['Turkey'],
 'Karachi': ['Pakistan'],
 'Bangalore': ['India'],
 'Ho Chi Minh City': ['Vietnam'],
 'Osaka': ['Japan'],
 'Chengdu': ['China'],
 'Tehran': ['Iran'],
 'Kinshasa': ['Congo (Kinshasa)'],
 'Rio de Janeiro': ['Brazil'],
 'Chennai': ['India'],
 'Xi’an': ['China'],
 'Lahore': ['Pakistan'],
 'Chongqing': ['China'],
 'Los Angeles': ['United States', 'Chile'],
